In [1]:
import requests
import re
import pandas as pd
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from retrying import retry
import random
import time
import emoji

ua = UserAgent()

headers = {
    "user-agent": ua.random,   
    "cookie": "ka_sessionid=007c0ea4fa6fa02ca9285e4a6ab4f08d; _ga=GA1.2.845049904.1668501166; __Host-KAGGLEID=CfDJ8M8bzImQuQVBugV8xw6Jv2vz4D6IJmy5bZh_Ch9KTzXmos9PJYvn-9bI2gqmIGHETjK6rtoQIZpKEvHIKEQVh2pis-Q8RfmDe8GBBFToY7M6gX-MeU0DOeuD; CSRF-TOKEN=CfDJ8Gm7kp8NARJBjtS2astAKvSLOjDUOhO9z4XDb4Cs9Gt-fy7xDzbswtX5Y6kW8oWViWeP7FKF5V1fsGgfMC7Y1tWQMgK-463lzixwzHf15g; GCLB=CMaXzdfs0Z6gfA; _gid=GA1.2.660380570.1669293594; XSRF-TOKEN=CfDJ8Gm7kp8NARJBjtS2astAKvRLudml_uJExERBlejV-5z-k3tc_OnNSCtTAqMaScV9hLrT-SgDHFAlPZpEIj3CpEz5zxR81FFsm-8Ksgh8Luesgt0CLytPG_5XXaqz5JVlzQuClGMZoB3iomwgwqdmFJ0; CLIENT-TOKEN=eyJhbGciOiJub25lIiwidHlwIjoiSldUIn0.eyJpc3MiOiJrYWdnbGUiLCJhdWQiOiJjbGllbnQiLCJzdWIiOiJ5aW5ncWlhbmMiLCJuYnQiOiIyMDIyLTExLTI0VDEyOjQxOjAzLjUwODI3MDlaIiwiaWF0IjoiMjAyMi0xMS0yNFQxMjo0MTowMy41MDgyNzA5WiIsImp0aSI6IjA2YjRkZmY4LTViZDAtNDE3NS1hNWM3LWY3MmRhNzNkMTYyMSIsImV4cCI6IjIwMjItMTItMjRUMTI6NDE6MDMuNTA4MjcwOVoiLCJ1aWQiOjk5MDcwOTAsImRpc3BsYXlOYW1lIjoiWWluZ3FpYW5DIiwiZW1haWwiOiJ5aW5ncWlhbmNoZW4uY25AZ21haWwuY29tIiwidGllciI6Ik5vdmljZSIsInZlcmlmaWVkIjpmYWxzZSwicHJvZmlsZVVybCI6Ii95aW5ncWlhbmMiLCJ0aHVtYm5haWxVcmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20va2FnZ2xlLWF2YXRhcnMvdGh1bWJuYWlscy9kZWZhdWx0LXRodW1iLnBuZyIsImZmIjpbIktlcm5lbHNEcmFmdFVwbG9hZEJsb2IiLCJLZXJuZWxzRmlyZWJhc2VQcm94eSIsIktlcm5lbHNHQ1NVcGxvYWRQcm94eSIsIktlcm5lbHNGaXJlYmFzZUxvbmdQb2xsaW5nIiwiS2VybmVsc1N0YWNrT3ZlcmZsb3dTZWFyY2giLCJDb21tdW5pdHlLbUltYWdlVXBsb2FkZXIiLCJUUFVDb21taXRTY2hlZHVsaW5nIiwiQWxsb3dGb3J1bUF0dGFjaG1lbnRzIiwiS2VybmVsc1NhdmVDZWxsT3V0cHV0IiwiS01MZWFybkRldGFpbCIsIkZyb250ZW5kQ29uc29sZUVycm9yUmVwb3J0aW5nIiwiUGhvbmVWZXJpZnlGb3JDb21tZW50cyIsIlBob25lVmVyaWZ5Rm9yTmV3VG9waWMiLCJMaWhwTmV4dFN0ZXBzIiwiTGlocE5leHRTdGVwc01ldHJpY3MiLCJLbUNvbXBzU3VibWlzc2lvblBhZ2UiLCJMZWFybkd1aWRlcyIsIkRhdGFzZXRzTWV0YWRhdGFUb0RhdGFUYWIiLCJEYXRhc2V0c1ZhbGlkYXRlVXBsb2FkZWRaaXBGaWxlcyIsIktlcm5lbEVkaXRvckhhbmRsZU1vdW50T25jZSJdLCJmZmQiOnsiS2VybmVsRWRpdG9yQXV0b3NhdmVUaHJvdHRsZU1zIjoiMzAwMDAiLCJGcm9udGVuZEVycm9yUmVwb3J0aW5nU2FtcGxlUmF0ZSI6IjAuMDEiLCJFbWVyZ2VuY3lBbGVydEJhbm5lciI6InsgfSIsIkNsaWVudFJwY1JhdGVMaW1pdCI6IjQwIiwiRmVhdHVyZWRDb21tdW5pdHlDb21wZXRpdGlvbnMiOiIzNTMyNSwzNzE3NCwzMzU3OSwzNzg5OCwzNzM1NCwzNzk1OSIsIkFkZEZlYXR1cmVGbGFnc1RvUGFnZUxvYWRUYWciOiJkYXRhc2V0c01hdGVyaWFsRGV0YWlsIn0sInBpZCI6ImthZ2dsZS0xNjE2MDciLCJzdmMiOiJ3ZWItZmUiLCJzZGFrIjoiQUl6YVN5QTRlTnFVZFJSc2tKc0NaV1Z6LXFMNjU1WGE1SkVNcmVFIiwiYmxkIjoiYWNiOWIxZTBkNjkxMGRkNzJjZDgyN2VhMGM4NzllZGYzYmJhOWY5ZSJ9.; _gat_gtag_UA_12629138_1=1",
    "x-xsrf-token": "CfDJ8Gm7kp8NARJBjtS2astAKvRLudml_uJExERBlejV-5z-k3tc_OnNSCtTAqMaScV9hLrT-SgDHFAlPZpEIj3CpEz5zxR81FFsm-8Ksgh8Luesgt0CLytPG_5XXaqz5JVlzQuClGMZoB3iomwgwqdmFJ0"
}

result = []

@retry(stop_max_attempt_number=10, wait_random_min=1000, wait_random_max=2000)
def get_info(resp):
    base_url = 'https://www.kaggle.com/competitions/'
    for i in resp['competitions']:
        try:
            comp = {}
            url = base_url + i['competitionName']
            html = requests.get(url, headers=headers).text

            comp['title'] = i['title']
            comp['url'] = url
            comp['briefDescription'] = i['briefDescription']
            
            try:
                comp['coverImageUrl'] = i['coverImageUrl']
            except:
                comp['coverImageUrl'] = None
                
            try:
                tag_info = re.findall('"tags":(.*?)]', html, re.S)[0]
                tag = re.findall('"name":"(.*?)",', tag_info, re.S)
                tag = ', '.join(tag)
            except:
                tag = None
            comp['tag'] = tag
          
            des_info = re.findall('"pages":(.*)</script>', html, re.S)[0]
            des = re.findall('"content":"(.*)",.*?"name":"[dD]escription"', des_info, re.S)[0]
            while des.find('"content":') != -1:
                index = des.find('"content"')
                des = re.findall('"content":"(.*)', des[index:], re.S)[0]
            if des.find('","') != -1:
                des = des[:des.find('","')]
            des = des.encode().decode("unicode_escape")
            des =  BeautifulSoup(des,'lxml').text.replace('\n', '')
            comp['description'] = des
            
            print('正在爬取{}'.format(i['competitionName']))
            result.append(comp)
        except:
            print(url)

if __name__ == '__main__':
    t = random.random()
    url = "https://www.kaggle.com/api/i/competitions.CompetitionService/ListCompetitions"
    data = '{"selector":{"competitionIds":[],"listOption":"LIST_OPTION_DEFAULT","sortOption":"SORT_OPTION_NEWEST","hostSegmentIdFilter":0,"searchQuery":"","prestigeFilter":"PRESTIGE_FILTER_UNSPECIFIED","participationFilter":"PARTICIPATION_FILTER_UNSPECIFIED","tagIds":[],"requireSimulations":false},"pageToken":"","pageSize":20,"readMask":"competitions,totalResults"}'
    resp = requests.post(url, data, headers = headers).json()
    get_info(resp)
    time.sleep(t)
    for i in range(1, 28):
        data = '{"selector":{"competitionIds":[],"listOption":"LIST_OPTION_DEFAULT","sortOption":"SORT_OPTION_NEWEST","hostSegmentIdFilter":0,"searchQuery":"","prestigeFilter":"PRESTIGE_FILTER_UNSPECIFIED","participationFilter":"PARTICIPATION_FILTER_UNSPECIFIED","tagIds":[],"requireSimulations":false},'+'"pageToken":"{}",'.format(i*20)+'"pageSize":20,"readMask":"competitions,totalResults"}'''
        resp = requests.post(url, data, headers = headers).json()
        get_info(resp)
        time.sleep(t)
    print(result)
    pd.DataFrame(result).to_csv('kaggle_competition.csv', index = None)

正在爬取otto-recommender-system
正在爬取tabular-playground-series-nov-2022
正在爬取scrabble-player-rating
正在爬取nfl-big-data-bowl-2023
正在爬取kaggle-survey-2022
正在爬取g2net-detecting-continuous-gravitational-waves
正在爬取lux-ai-2022-beta
正在爬取tabular-playground-series-oct-2022
正在爬取novozymes-enzyme-stability-prediction
正在爬取tabular-playground-series-sep-2022
正在爬取feedback-prize-english-language-learning
正在爬取open-problems-multimodal
正在爬取ai-village-ctf
正在爬取big-data-derby-2022
正在爬取tabular-playground-series-aug-2022
正在爬取dfl-bundesliga-data-shootout
正在爬取rsna-2022-cervical-spine-fracture-detection
正在爬取google-universal-image-embedding
正在爬取mayo-clinic-strip-ai
正在爬取tabular-playground-series-jul-2022
正在爬取hubmap-organ-segmentation
正在爬取tabular-playground-series-jun-2022
正在爬取amex-default-prediction
正在爬取feedback-prize-effectiveness
正在爬取AI4Code
正在爬取smartphone-decimeter-2022
正在爬取tabular-playground-series-may-2022
正在爬取uw-madison-gi-tract-image-segmentation
正在爬取foursquare-location-matching
正在爬取kore-2022
正在爬取jpx-tokyo-stock-excha

正在爬取new-york-city-taxi-fare-prediction
正在爬取tgs-salt-identification-challenge
正在爬取costa-rican-household-poverty-prediction
正在爬取google-ai-open-images-visual-relationship-track
正在爬取google-ai-open-images-object-detection-track
正在爬取demand-forecasting-kernels-only
正在爬取flavours-of-physics-kernels-only
正在爬取whats-cooking-kernels-only
正在爬取movie-review-sentiment-analysis-kernels-only
正在爬取forest-cover-type-kernels-only
正在爬取santander-value-prediction-challenge
正在爬取youtube8m-2018
正在爬取home-credit-default-risk
正在爬取trackml-particle-identification
正在爬取avito-demand-prediction
正在爬取cvpr-2018-autonomous-driving
正在爬取imaterialist-challenge-fashion-2018
正在爬取freesound-audio-tagging
正在爬取imaterialist-challenge-furniture-2018
正在爬取talkingdata-adtracking-fraud-detection
正在爬取donorschoose-application-screening
正在爬取inaturalist-2018
正在爬取womens-machine-learning-competition-2018
正在爬取mens-machine-learning-competition-2018
正在爬取competitive-data-science-predict-future-sales
正在爬取landmark-retrieval-challenge
正在爬取landmark-recogn

正在爬取bluebook-for-bulldozers
正在爬取event-recommendation-engine-challenge
正在爬取leapfrogging-leaderboards
正在爬取traveling-santa-problem
正在爬取visualize-the-state-of-education-in-colorado
正在爬取RxVolumePrediction
正在爬取flight
正在爬取hospital
正在爬取facebook-ii
正在爬取DarkWorlds
https://www.kaggle.com/competitions/titanic
正在爬取detecting-insults-in-social-commentary
正在爬取cir-prospect
正在爬取customer-retention
正在爬取GEF2012-wind-forecasting
正在爬取global-energy-forecasting-competition-2012-load-forecasting
正在爬取us-census-challenge
正在爬取predict-closed-questions-on-stack-overflow
正在爬取harvard-business-review-vision-statement-prospect
正在爬取acm-sf-chapter-hackathon-small
正在爬取acm-sf-chapter-hackathon-big
正在爬取MerckActivity
正在爬取job-recommendation
正在爬取digit-recognizer
正在爬取MusicHackathon
正在爬取Raising-Money-to-Fund-an-Organizational-Mission
正在爬取pf2012-diabetes
正在爬取cprod1
正在爬取asap-sas
正在爬取predict-wordpress-likes
正在爬取emc-data-science
正在爬取pf2012-at
正在爬取pf2012
正在爬取FacebookRecruiting
正在爬取twitter-psychopathy-prediction
正在爬取twitter-personality

In [2]:
df = pd.read_csv('kaggle_competition.csv')
df

,title,url,briefDescription,coverImageUrl,tag,description
0,OTTO – Multi-Objective Recommender System,https://www.kaggle.com/competitions/otto-recom...,Build a recommender system based on real-world...,https://storage.googleapis.com/kaggle-competit...,"retail and shopping, recommender systems, weig...",## Goal of the CompetitionThe goal of this com...
1,Tabular Playground Series - Nov 2022,https://www.kaggle.com/competitions/tabular-pl...,Practice your ML skills on this approachable d...,https://storage.googleapis.com/kaggle-competit...,"tabular, ensembling, logloss",You may have heard that blending predictions f...
2,Scrabble Player Rating,https://www.kaggle.com/competitions/scrabble-p...,Predict players' ratings based on Woogles.io g...,https://storage.googleapis.com/kaggle-competit...,"tabular, rmse",Are you a Kaggle Scrabble Grandmaster? In the ...
3,NFL Big Data Bowl 2023,https://www.kaggle.com/competitions/nfl-big-da...,Help evaluate linemen on pass plays,https://storage.googleapis.com/kaggle-competit...,NaN,##Goal of the CompetitionThe National Football...
4,2022 Kaggle Machine Learning & Data Science Su...,https://www.kaggle.com/competitions/kaggle-sur...,The most comprehensive dataset available on th...,https://storage.googleapis.com/kaggle-competit...,"data analytics, online communities, survey ana...",**Welcome to Kaggle's annual Machine Learning ...
...,...,...,...,...,...,...
543,INFORMS Data Mining Contest 2010,https://www.kaggle.com/competitions/informs2010,The goal of this contest is to predict short t...,NaN,auc,The INFORMS Data Mining Section (in conjun...
544,World Cup 2010 - Confidence Challenge,https://www.kaggle.com/competitions/worldcupconf,The Confidence Challenge requires competitors ...,NaN,custom metric,We are also running a World Cup 2010 - Take on...
545,World Cup 2010 - Take on the Quants,https://www.kaggle.com/competitions/worldcup2010,Quants at Goldman Sachs and JP Morgan have mod...,NaN,custom metric,Can you outdo the quants?As a break from proje...
546,Predict HIV Progression,https://www.kaggle.com/competitions/hivprogres...,This contest requires competitors to predict t...,NaN,mce,"According to the World Health Organization, HI..."
